# 서울스토어 크롤러4

분류용 상품 이미지를 크롤링하기 위한 메타 정보 크롤러입니다. a.k.a "통" 만들기 사전작업

서울스토어의 카테고리별 상품 정보(이미지, 상세정보 링크)를 크롤링하는 크롤러입니다. 크롤링 결과를 활용해서 상품 메타정보를 저장 + 상품 이미지(별도의 크롤러 활용)를 저장합니다.

크롤링한 카테고리는 아래와 같습니다.
- 상의 (super_category: 0)
    - 티셔츠 (category: 0)
        - 롱슬리브 (sub_category: 0)
        - 숏슬리브 (sub_category: 1)
        - 슬리브리스 (sub_category: 2)
        - 크롭탑 (sub_category: 3)
        - 폴로 셔츠 (sub_category: 4)
    - 후디/스웨트셔츠 (category: 1)
        - 후디 (sub_category: 5)
        - 스웨트셔츠 (sub_category: 6)
        - 집업후디 (sub_category: 7)
    - 셔츠/블라우스 (category: 2)
        - 롱 슬리브 (sub_category: 8)
        - 숏 슬리브 (sub_category: 9)
        - 블라우스 (sub_category: 10)
    - 니트웨어 (category: 3)
        - 라운드넥 (sub_category: 11)
        - 브이넥 (sub_category: 12)
        - 터틀넥 (sub_category: 13)
        - 베스트 (sub_category: 14)
        - 가디건 (sub_category: 15)
- 하의 (super_category: 1)
    - 스커트 (category: 4)
        - 미니 (sub_category: 16)
        - 미디/롱 (sub_category: 17)
    - 팬츠 (category: 5)
        - 치노 (sub_category: 18)
        - 스웨트팬츠 (sub_category: 19)
        - 스트레이트 (sub_category: 20)
        - 와이드 (sub_category: 21)
        - 스키니  (sub_category: 22)
        - 부츠컷 (sub_category: 23)
        - 쇼츠 (sub_category: 24)
        - 레깅스 (sub_category: 25)
    - 데님 (category: 6)
        - 스트레이트 (sub_category: 26)
        - 와이드 (sub_category: 27)
        - 스키니 (sub_category: 28)
        - 부츠컷 (sub_category: 29)
        - 크롭 (sub_category: 30)
        - 스커트 (sub_category: 31)
        - 쇼츠 (sub_category: 32)
- 원피스 (super_category: 2)
    - 원피스 (category: 7)
        - 미니 (sub_category: 33)
        - 미디/맥시 (sub_category: 34)
        - 드레스 (sub_category: 35)
    - 오버올 (category: 8)
        - 올인원 (sub_category: 36)
        - 점프수트 (sub_category: 37)

크롤링 결과는

`{"key": 고유번호(5자리수), 
"product_url": 상품url, 
"img_url": 이미지url, 
"super_category": 상의0, 하의1, 원피스2, 
"category": 티셔츠0, ..., 오버올8, 
"sub_category": 롱슬리브0, ..., 점프수트37} `

형태의 json 파일로 저장합니다.

In [1]:
from __future__ import unicode_literals
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import glob
import json
import os
import re
import sys
import time
import traceback
from builtins import open
from time import sleep

In [3]:
#카테고리별 url(상의, 하의, 원피스)
root_urls = ['https://www.seoulstore.com/categories/1251/regDatetime/desc',
            'https://www.seoulstore.com/categories/1253/regDatetime/desc',
            'https://www.seoulstore.com/categories/1252/regDatetime/desc']

In [59]:
category_dict = {
    1002: {"super_category": 0, "category": 0, "sub_category": 0, "name": "롱슬리브"},
    1003: {"super_category": 0, "category": 0, "sub_category": 1, "name": "숏슬리브"},
    1004: {"super_category": 0, "category": 0, "sub_category": 2, "name": "슬리브리스"},
    1005: {"super_category": 0, "category": 0, "sub_category": 3, "name": "크롭 탑"},
    1006: {"super_category": 0, "category": 0, "sub_category": 4, "name": "폴로 셔츠"},
    1008: {"super_category": 0, "category": 1, "sub_category": 5, "name": "후디"},
    1010: {"super_category": 0, "category": 1, "sub_category": 6, "name": "스웨트셔츠"},
    1009: {"super_category": 0, "category": 1, "sub_category": 7, "name": "집업후디"},
    1012: {"super_category": 0, "category": 2, "sub_category": 8, "name": "롱 슬리브"},
    1013:{"super_category": 0, "category": 2, "sub_category": 9, "name": "숏 슬리브"},
    1014: {"super_category": 0, "category": 2, "sub_category": 10, "name": "블라우스"},
    1016: {"super_category": 0, "category": 3, "sub_category": 11, "name": "라운드넥" },
    1017: {"super_category": 0, "category": 3, "sub_category": 12, "name": "브이넥"},
    1018: {"super_category": 0, "category": 3, "sub_category": 13, "name": "터틀넥"},
    1019: {"super_category": 0, "category": 3, "sub_category": 14, "name": "베스트"},
    1020: {"super_category": 0, "category": 3, "sub_category": 15, "name": "가디건"},
    
    1025: {"super_category": 1, "category": 4, "sub_category": 16, "name": "미니"},
    1026: {"super_category": 1, "category": 4, "sub_category": 17, "name": "미디/롱"},
    1028: {"super_category": 1, "category": 5, "sub_category": 18, "name": "치노"},
    1034: {"super_category": 1, "category": 5, "sub_category": 19, "name": "스웨트팬츠"},
    1031: {"super_category": 1, "category": 5, "sub_category": 20, "name": "스트레이트"},
    1032: {"super_category": 1, "category": 5, "sub_category": 21, "name": "와이드"},
    1030: {"super_category": 1, "category": 5, "sub_category": 22, "name": "스키니"},
    1033: {"super_category": 1, "category": 5, "sub_category": 23, "name": "부츠컷"},
    1029: {"super_category": 1, "category": 5, "sub_category": 24, "name": "쇼츠"},
    1035: {"super_category": 1, "category": 5, "sub_category": 25, "name": "레깅스"},
    1040: {"super_category": 1, "category": 6, "sub_category": 26, "name": "스트레이트"},
    1041: {"super_category": 1, "category": 6, "sub_category": 27, "name": "와이드"},
    1039: {"super_category": 1, "category": 6, "sub_category": 28, "name": "스키니"},
    1042: {"super_category": 1, "category": 6, "sub_category": 29, "name": "부츠컷"},
    1043: {"super_category": 1, "category": 6, "sub_category": 30, "name": "크롭"},
    1038: {"super_category": 1, "category": 6, "sub_category": 31, "name": "스커트"},
    1037: {"super_category": 1, "category": 6, "sub_category": 32, "name": "쇼츠"},
    
    1022: {"super_category": 2, "category": 7, "sub_category": 33, "name": "미니"},
    1023: {"super_category": 2, "category": 7, "sub_category": 34, "name": "미디/맥시"},
    1073: {"super_category": 2, "category": 7, "sub_category": 35, "name": "드레스"},
    1045: {"super_category": 2, "category": 8, "sub_category": 36, "name": "올인원"},
    1046: {"super_category": 2, "category": 8, "sub_category": 37, "name": "점프수트"}
}

In [4]:
category_dict = {
    1251: {1001: {1002, 1003, 1004, 1005, 1006}, 
           1007: {1008, 1010, 1009}, 
           1011: {1012, 1013, 1014}, 
           1015: {1016, 1017, 1018, 1019, 1020}}, 
    1253: {1024: {1025, 1026},
           1027: {1028, 1034, 1031, 1032, 1030, 1033, 1029, 1035},
           1036: {1040, 1041, 1039, 1042, 1043, 1038, 1037}},
    1252: {1021: {1022, 1023, 1073},
           1044: {1045, 1046}}
}

In [27]:
category_dict

{1251: {1001: {1002, 1003, 1004, 1005, 1006},
  1007: {1008, 1009, 1010},
  1011: {1012, 1013, 1014},
  1015: {1016, 1017, 1018, 1019, 1020}},
 1253: {1024: {1025, 1026},
  1027: {1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035},
  1036: {1037, 1038, 1039, 1040, 1041, 1042, 1043}},
 1252: {1021: {1022, 1023, 1073}, 1044: {1045, 1046}}}

In [22]:
for k in category_dict:
    for k2 in category_dict[k]:
        print(category_dict[k][k2])

{1002, 1003, 1004, 1005, 1006}
{1008, 1009, 1010}
{1012, 1013, 1014}
{1016, 1017, 1018, 1019, 1020}
{1025, 1026}
{1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035}
{1037, 1038, 1039, 1040, 1041, 1042, 1043}
{1073, 1022, 1023}
{1045, 1046}


In [62]:
def get_products(category_dict, num, filepath, wait_time):
    
    """
    Arguments:
    - category_dict: dict. 카테고리 고유번호: 카테고리 정보 키밸류 페어를 원소로 함
    - num: int. url당 크롤링할 아이템 수
    - filepath: str. 크롤링 결과를 저장할 json 파일 경로
    - wait_time: int. 로딩을 몇 초까지 기다릴지
    
    Return:
    - 없음
    
    크롤링 완료 후 "key":고유번호(5자리수)를 추가해줄 것임 (key를 사후에 추가하는 이유는 중복 크롤링 처리를 쉽게 하기 위함임)
    """
    product_set = set()   #중복 크롤링 거르기 위한 셋. product_url을 원소로 함
    
    browser = webdriver.Chrome('chromedriver')   #크롬 브라우저 실행
    wait = WebDriverWait(browser, wait_time)
    
    for cat in category_dict:
        url = 'https://www.seoulstore.com/categories/{}/regDatetime/desc'.format(str(cat))
        browser.get(url)
        body = browser.find_element_by_tag_name('body')

        count = 0    #더 이상 로드되는 데이터가 없을 시 크롤링 종료하기 위해 필요한 count임
        prev_posts_count = 0
        ele_posts = browser.find_elements_by_class_name('image_container')

        ##########추가한 부분###########
        while len(ele_posts) < num:
            body.send_keys(Keys.PAGE_DOWN)
            ele_posts = browser.find_elements_by_class_name('image_container')
#             if len(ele_posts) % 100 == 0:
#                 print(len(ele_posts))

            cur_posts_count = len(ele_posts)
            if prev_posts_count == cur_posts_count:
                count += 1
            else: count = 0
            if count > 50:
                    break

            prev_posts_count = cur_posts_count

        ##########추가한 부분 끝##########
        cat_post_count = 0   #카테고리별 크롤링된 아이템 수 세기
        for ele in ele_posts:
            product_url= ele.find_element_by_tag_name('a').get_attribute('href')
            if product_url not in product_set:
                cat_post_count +=1
                dict_post = { "product_url": product_url }
                wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'ImageLoader.ratio_1_1.loaded')))
                ele_img = ele.find_element_by_class_name('ImageLoader.ratio_1_1.loaded')
                dict_post["img_url"] = ele_img.get_attribute("src")
                dict_post["sub_category"] = category_dict[cat]["sub_category"]
                dict_post["category"] = category_dict[cat]["category"]
                dict_post["super_category"] = category_dict[cat]["super_category"]
                
                product_set.add(product_url)

                out = json.dumps(dict_post, ensure_ascii=False)    #json 형식으로 정보 변환
                out += ', '    #아이템 정보 분류하기 위해 끝에 쉼표 추고
                with open(filepath, "a", encoding="utf-8") as f:
                    f.write(out)
        
        print("saved {} items from {} section".format(cat_post_count, category_dict[cat]['name']))
    
    browser.close()

In [63]:
filepath = 'categorized_tong.json'

if not filepath:  #파일 없을 경우 만들어주기
    open(filepath, 'a').close()
        
#크롤링
get_products(category_dict, 1, filepath, 300)

saved 12 items from 롱슬리브 section
saved 12 items from 숏슬리브 section
saved 12 items from 슬리브리스 section
saved 12 items from 크롭 탑 section
saved 12 items from 폴로 셔츠 section
saved 12 items from 후디 section
saved 12 items from 스웨트셔츠 section
saved 12 items from 집업후디 section
saved 12 items from 롱 슬리브 section
saved 12 items from 숏 슬리브 section
saved 12 items from 블라우스 section
saved 12 items from 라운드넥 section
saved 12 items from 브이넥 section
saved 12 items from 터틀넥 section
saved 12 items from 베스트 section
saved 11 items from 가디건 section
saved 12 items from 미니 section
saved 12 items from 미디/롱 section
saved 12 items from 치노 section
saved 11 items from 스웨트팬츠 section
saved 10 items from 스트레이트 section
saved 10 items from 와이드 section
saved 12 items from 스키니 section
saved 11 items from 부츠컷 section
saved 11 items from 쇼츠 section
saved 12 items from 레깅스 section
saved 11 items from 스트레이트 section
saved 11 items from 와이드 section
saved 12 items from 스키니 section
saved 12 items from 부츠컷 section
saved 10 items from 크

In [54]:
with open('../categorized_tong.json', 'r', encoding = 'utf-8') as f:
    file = f.read()

len(file.split('},'))

442